# Trade
- Swing trading/long-term trading
    - Exposed to overnight risk (close price previous day might not equal to open 
    price next day if there are major events happening between market closure and
    market open).
- Assume I already have which day to long, which day to short
- Conduct post-trade analysis
- Refine risk management techniques (Comparing starting on 2023-12-22)
    - Boeing: Main character in the events
        - Stock -18.61%

    - Direct competitors
        - Airbus (EPA: AIR): Boeing's primary competitor in commercial aircraft manufacturing
            - Stock +5.93%
        - Lockhead Martin (LMT): More focused on defense but also compete in aerospace
    - Suppliers
        - General Electric (GE): Supplies engines for Boeing aircraft
            - Have presence in aviation, healthcare, power, renewable energy
            - Doesn't seem to be affected
            - Can also supply engines to other aircraft manufacturers (effect on
            stock price is complicated)
    - Customers
        - Alaska Airlines (ALK): Main airline involved
            - Stock -11.73%
        - American Airlines (UAL - NasdaqGS)
            - Stock -4.91%
        - Delta Air Lines (DAL)
            - -11.73%
        - Southwest Airlines
- Trading timing (NYSE) vs news timing
    - The news was updated on January 18, 2024, at 4:36 AM GMT+8, which translates to January 17, 2024, at 3:36 PM Eastern Time (since GMT+8 is 13 hours ahead of Eastern Time). Since the NYSE closes at 4:00 PM ET, this news would have come out just before the market close.
    - Difference stock exchanges might operate at different timings also
- No training and validation - straight go to validation (backtesting)



# Set Up

In [30]:
import os
import ast
import requests
import logging

import yfinance as yf
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np
import finnhub
from dotenv import load_dotenv
from pathlib import Path    
import sys
import time
sys.path.append('../') # Change the python path at runtime

# Self-created modules
from src.utils import path as path_yq

In [31]:
load_dotenv()
POLYGON_API_KEY = os.environ.get('POLYGON_API_KEY')

BACKTEST_START_DATE = '2024-01-01'
BACKTEST_START_STRING = '20240101'
BACKTEST_END_DATE = '2024-01-14'
BACKTEST_END_STRING = '20240114'

cur_dir = Path.cwd()
root_dir = path_yq.get_root_dir(cur_dir)

logging.basicConfig(filename=Path.joinpath(root_dir, 'logs', 'trading_system.log'),
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.DEBUG)

# Fetch Tick Data

## Polygon

Polygon docs: https://polygon.io/docs/stocks/get_v2_aggs_ticker__stocksticker__range__multiplier___timespan___from___to

- FIXME: The timings include those in pre-market hours
- The timestamp is in ms, not sec

Similar to download data codes
TODO: Assumption: assume other stocks share the same timezone

- The data is incomplete (not every minute)
24265	47739.0	217.6996	217.6800	217.7000	217.7000	217.6800	1705096560000	21	2024-01-12 21:56:00
24266	171.0	217.5423	217.5000	217.5000	217.5000	217.5000	1705096800000	5	2024-01-12 22:00:00

In [ ]:
url = f"https://api.polygon.io/v2/aggs/ticker/BA/range/1/minute/{BACKTEST_START_DATE}/{BACKTEST_END_DATE}?adjusted=true&sort=asc&limit=50000&apiKey={POLYGON_API_KEY}"

# Make the GET request
resp = requests.get(url)

In [ ]:
# Check if the request was successful
if resp.status_code == 200:
    # Convert the 'results' list to a DataFrame
    df = pd.DataFrame(resp.json().get('results'))

    # Rename the columns to more descriptive names
    column_mapping = {
        "v": "Volume",
        "vw": "VWAP",
        "o": "Open",
        "c": "Close",
        "h": "High",
        "l": "Low",
        "t": "Timestamp",
        "n": "Transactions"
        # Add more mappings as necessary
    }

    df.rename(columns=column_mapping, inplace=True)

    # Optionally, convert the 'Timestamp' column from Unix milliseconds to a datetime format
    df['Datetime'] = pd.to_datetime(df['Timestamp'], unit='ms')

    # Display the updated DataFrame
    print(df)
else:
    # Handle errors (e.g., logging, raising an exception)
    print(f"Error fetching data: {resp.status_code}, {resp.text}")



In [ ]:
# Boeing open high low close data
raw_path = Path.joinpath(root_dir, 'data', 'raw', 'BA_OHLC_20231101_to_20240131.csv')
df.to_csv(raw_path, index=False)

## Yahoo

In [ ]:
# Define the ticker list
ticker_list = ['BA']

# Fetch the data
dl_data = yf.download(ticker_list, start=BACKTEST_START_DATE, end=BACKTEST_END_DATE) # Auto adjust is false

dl_data = pd.DataFrame(dl_data)
data = dl_data.drop(columns=['Close'], axis=1)
data = data.rename(columns={'Adj Close': 'Close'})
display(data.isna().sum(axis=0)) # Axis=0: along the indices, row-wise opertaion
# Gives the sum for rows in a column
data.index = pd.to_datetime(data.index)
data


In [ ]:
dates = pd.DataFrame(data.index.strftime('%Y-%m-%d'))
# dates.to_csv("trading_dates.csv", index=False)

In [ ]:
# After performing sentiment
stm_path = root_dir.joinpath('data', 'proc', 'boeing_stm_20231101_to_20240131.csv')
news = pd.read_csv(stm_path, index_col=False)
news2 = news[['datetime2', 'news_pol_blob']]
news2

news2.plot()
# # data['Sentiment'] = np.random.random(len(data)) * 2 - 1
# display(len(data))
# sentiment = np.array([0, -1, -0.8, 0, 0, 0]) # Put -1 on 01-05 (Before the whole thing Boeing case appeared after market closed on 01-05 to prepare to trade for 01-08)
# data['Sentiment'] = sentiment
# display(data.tail(20))

In [ ]:
# Ensure datetime2 in news2 is in pandas datetime format
news2['datetime2'] = pd.to_datetime(news2['datetime2'])

# Assuming data.index is already a DatetimeIndex, no need to convert it again
# Just ensure it's sorted
data.sort_index(inplace=True)

# Function to find the closest previous date in data for each date in news2
def find_closest_previous_date(target_date, date_index):
    previous_dates = date_index[date_index <= target_date]
    if not previous_dates.empty:
        return previous_dates.max()
    else:
        return pd.NaT  # Return Not-A-Time (NaT) if no previous date is found

# Apply the function to each date in news2['datetime2']
closest_dates = news2['datetime2'].apply(lambda x: find_closest_previous_date(x, data.index))

# Add this closest date information to news2
news2['closest_date'] = closest_dates
news2

In [ ]:
# TODO: Need to think of how to combine the data (might have many neutral etc.)
# as_index will retain closest_date
news3 = news2.groupby('closest_date', as_index=False)['news_pol_blob'].mean().reset_index(drop=True) 
news3

In [ ]:
merged = pd.merge(data, news3, left_on='Date', right_on='closest_date', how='left')
merged

In [ ]:
# Clean for 2 lines only
merged2 = merged.dropna().reset_index(drop=True)
merged2

# Merge data

In [4]:

def convert_data(row):
    """
    A function from sentiment.ipynb.
    """
    try:
        # First, try to evaluate the row as a list
        evaluated = ast.literal_eval(row)
        # If the result is a list, return it directly
        if isinstance(evaluated, list):
            return evaluated
        # If not, it's already the correct type (int, float, etc.)
        return evaluated
    except ValueError:
        # Handle the case where the row is not a valid Python literal
        # This could be a string that should not be converted
        return row
    except SyntaxError:
        # Handle syntax errors which might occur if ast.literal_eval can't parse the string
        return row
    except Exception as e:
        print(f'Exception: {e}')
        return row

# TODO:
score_path = root_dir.joinpath('data', 'proc', f'boeing_score_{BACKTEST_START_STRING}_to_{BACKTEST_END_STRING}.csv') # TODO: Change dates
df9 = pd.read_csv(score_path, index_col=False)

# Apply the conversion function to each specified column
for col in df9.columns:
    df9[col] = df9[col].apply(convert_data)
df9['datetime2'] = pd.to_datetime(df9['datetime2'])

# print(df8.equals(df7))
# print(type(df8['datetime2'][0]))

In [5]:
# Boeing open high low close data
raw_path = Path.joinpath(root_dir, 'data', 'raw', f'BA_OHLC_20231101_to_20240131.csv')
tick = pd.read_csv(raw_path, index_col=False)
tick = tick.sort_values(by='Datetime')
tick['Datetime'] = pd.to_datetime(tick['Datetime'])
tick = tick[(tick['Datetime'] >= BACKTEST_START_DATE) & (tick['Datetime'] <= BACKTEST_END_DATE)]

In [6]:
tick[tick['Datetime'] >= '2024-01-12 21:47:23'].head()

,Volume,VWAP,Open,Close,High,Low,Timestamp,Transactions,Datetime
24265,47739.0,217.6996,217.6800,217.7000,217.7000,217.6800,1705096560000,21,2024-01-12 21:56:00
24266,171.0,217.5423,217.5000,217.5000,217.5000,217.5000,1705096800000,5,2024-01-12 22:00:00
24267,202.0,217.6949,217.6900,217.6999,217.6999,217.6900,1705096920000,3,2024-01-12 22:02:00
24268,119.0,217.6883,217.6892,217.6892,217.6892,217.6892,1705097280000,5,2024-01-12 22:08:00
24269,100.0,217.6900,217.6900,217.6900,217.6900,217.6900,1705097520000,1,2024-01-12 22:12:00


In [7]:
# Assuming data.index is already a DatetimeIndex, no need to convert it again
df9['datetime2'] = pd.to_datetime(df9['datetime2'])
tick['Datetime'] = pd.to_datetime(tick['Datetime'])

# Make sure to sort first
df9 = df9.sort_values(by='datetime2')
tick = tick.sort_values(by='Datetime')

# Function to find the closest previous date in tick for each date in news2
def find_closest_prev_date(target_date, date_col):
    # The information gotten at this time point can only be used in the next time point
    prev_dates = date_col[date_col <= target_date] 
    if not prev_dates.empty:
        return prev_dates.max()
    else:
        # Can happen when we have news on weekends but there is no more tick data (end of backtest)
        print("WARNING.")
        return pd.NaT  # Return Not-A-Time (NaT) if no previous date is found


# Apply the function to each date in news2['datetime2']
closest_dates = df9['datetime2'].apply(lambda x: find_closest_prev_date(x, tick['Datetime']))

# Add this closest date information to news2
df9['closest_date'] = closest_dates
df9.sort_values(by='datetime2')

,datetime2,cln_hdl,cln_smr,cln_news,cln_hdl_lemma,cln_smr_lemma,cln_news_lemma,cln_hdl_pol_blob,cln_smr_pol_blob,cln_news_pol_blob,...,cln_hdl_lemma_pol_blob_score,cln_smr_lemma_pol_blob_score,cln_news_lemma_pol_blob_score,cln_hdl_pol_sid_score,cln_smr_pol_sid_score,cln_news_pol_sid_score,cln_hdl_lemma_pol_sid_score,cln_smr_lemma_pol_sid_score,cln_news_lemma_pol_sid_score,closest_date
3,2024-01-02 12:10:00,[Bell-Boeing Secures Contract to Aid MV-22 Osp...,"[Bell-Boeing, a joint venture between Boeing (...","[Bell-Boeing, a joint venture (JV) between The...",[Bell-Boeing Secures Contract Aid MV-22 Osprey...,"[Bell-Boeing , joint venture Boeing ( BA ) Bel...","[Bell-Boeing , joint venture ( JV ) Boeing Com...",[0.0],[0.0],"[0.0, 0.0, -0.1, 0.0, 0.0, 0.0, 0.020833333333...",...,0.000000,0.000000,0.097336,0.3182,0.00000,0.426300,0.3182,0.00000,0.425179,2024-01-02 12:10:00
2,2024-01-02 13:48:00,[Boeing Stock Just Got Downgraded.],[Goldman Sachs analyst Noah Poponak took share...,[Goldman Sachs analyst Noah Poponak took share...,[Boeing Stock Got Downgraded .],[Goldman Sachs analyst Noah Poponak took share...,[Goldman Sachs analyst Noah Poponak took share...,[0.0],[0.0],[0.0],...,0.000000,0.000000,0.000000,0.0000,0.15310,0.153100,0.0000,0.29600,0.296000,2024-01-02 13:48:00
1,2024-01-02 17:17:13,[A loose bolt is the least of Boeings problems...,[Boeing (BA) shares tick down Tuesday morning ...,[Boeing (BA) shares tick down Tuesday morning ...,"[loose bolt least Boeings problem , analyst say]",[Boeing ( BA ) share tick Tuesday morning foll...,[Boeing ( BA ) share tick Tuesday morning foll...,[-0.18846153846153846],"[-0.07777777777777779, -0.07692307692307693, 0...","[-0.07777777777777779, -0.07692307692307693, 0...",...,-0.188462,0.068820,0.074968,-0.6124,-0.15644,0.017804,-0.6124,-0.07982,0.052892,2024-01-02 17:17:00
0,2024-01-02 19:19:53,"[US STOCKS-SP, Nasdaq start 2024 in subdued fa...",[The SP 500 and Nasdaq Composite dropped in th...,[Apple down on Barclays downgrade Tesla flat d...,"[US STOCKS-SP , Nasdaq start 2024 subdued fash...",[SP 500 Nasdaq Composite dropped first trading...,[Apple Barclays downgrade Tesla flat despite r...,[0.0],"[0.08333333333333333, 0.0]","[-0.034523809523809526, -0.17916666666666667, ...",...,0.000000,0.083333,0.054308,0.0000,-0.09760,0.194961,0.0000,-0.09760,0.186779,2024-01-02 19:19:00
8,2024-01-03 14:09:00,[RTX Secures a $345M Deal to Build StormBreake...,[RTX wins deal to manufacture the 10th lot of ...,[RTX Corp. RTX recently secured a modification...,[RTX Secures $ 345M Deal Build StormBreaker Mu...,[RTX win deal manufacture 10th lot StormBreake...,[RTX Corp. RTX recently secured modification c...,[0.0],[0.3],"[0.0, -0.45, -0.1, 0.0, 0.0, -0.1125, 0.0, 0.3...",...,0.000000,0.300000,0.068609,0.3182,0.57190,0.345733,0.3182,0.58590,0.352971,2024-01-03 14:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2024-01-12 20:40:54,[UPDATE 1-Delta orders up to 40 Airbus A350-10...,[Delta Air Lines on Friday unveiled an order f...,"[(New throughout, adds comments from Deltas CE...",[UPDATE 1-Delta order 40 Airbus A350-1000 wide...,[Delta Air Lines Friday unveiled order 40 new ...,"[( New throughout , add comment Deltas CEO par...",[0.0],"[0.19318181818181818, 0.125, 0.0]","[0.17424242424242423, 0.125, 0.0, 0.2047619047...",...,0.000000,0.159091,0.204149,0.0000,0.00000,0.366471,0.0000,0.00000,0.402729,2024-01-12 20:40:00
94,2024-01-12 21:06:00,"[Boeing Stock Is Down Again., The FAA Is Inten...",[Keeping up with the flow of news about the Al...,[Keeping up with the flow of news about the Al...,"[Boeing Stock ., FAA Intensifying Oversight .]",[Keeping flow news Alaska Airlines incident im...,[Keeping flow news Alaska Airlines incident im...,"[-0.15555555555555559, 0.0]",[0.2],[0.2],...,0.000000,0.200000,0.200000,0.0000,-0.37160,-0.371600,0.0000,-0.12800,-0.128000,2024-01-12 21:06:00
93,2024-01-12 21:08:00,"[Top Stock Reports for NVIDIA, Boeing Sony]",[Todays Resea

In [ ]:
# def drop_na(df):
#     # Drop all the news_content with na
#     print(f"Before dropping na: {df.isna().sum().sum()}")
#     df1 = df.dropna()
#     df1.reset_index(inplace=True, drop=True)
#     print(f"After dropping na: {df.isna().sum().sum()}")
#     return df1

# drop_na(df9).head()

In [64]:
COL = "cln_news_pol_sid_score"
# COL = "cln_news_pol_blob_score"
df10 = df9.groupby('closest_date', as_index=False)[COL].mean().reset_index(drop=True) 

merged2 = pd.merge(tick, df10, left_on='Datetime', right_on='closest_date', how='left')
# merged
merged2.reset_index(inplace=True, drop=True)


In [58]:
merged2[COL].describe()

count    94.000000
mean      0.098593
std       0.126931
min      -0.500000
25%       0.022669
50%       0.081047
75%       0.180368
max       0.500000
Name: cln_news_pol_blob_score, dtype: float64

In [49]:
merged2[merged2['Datetime'] >= pd.to_datetime('2024-01-03T14:17:00')]


,Volume,VWAP,Open,Close,High,Low,Timestamp,Transactions,Datetime,closest_date,cln_news_pol_sid_score
547,207.0,247.7027,247.70,247.7000,247.7000,247.7000,1704291420000,6,2024-01-03 14:17:00,2024-01-03 14:17:00,0.648005
548,308.0,247.7006,247.70,247.7000,247.7000,247.7000,1704291540000,8,2024-01-03 14:19:00,NaT,NaN
549,264.0,247.6105,247.60,247.6000,247.6000,247.6000,1704291780000,9,2024-01-03 14:23:00,NaT,NaN
550,1157.0,247.5724,247.60,247.5031,247.6001,247.5031,1704291840000,49,2024-01-03 14:24:00,NaT,NaN
551,563.0,247.5684,247.60,247.6000,247.6000,247.6000,1704291900000,19,2024-01-03 14:25:00,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5204,632.0,217.2991,217.30,217.3000,217.3000,217.3000,1705107000000,7,2024-01-13 00:50:00,NaT,NaN
5205,425.0,217.3412,217.35,217.3500,217.3500,217.3500,1705107060000,3,2024-01-13 00:51:00,NaT,NaN
5206,836.0,217.3015,217.30,217.3000,217.3000,217.3000,1705107300000,16,2024-01-13 00:55:00,NaT,NaN
5207,2492.0,217.0491,217.16,217.0200,217.1600,217.0000,1705107420000,52,2024-01-13 00:57:00,NaT,NaN


# Backtesting
- Pros
    - Test single strategy
    - Have optimizer, graphs
- Cons
    - Cannot trade multiple assets FIXME: not applicable to portfolio
    - Does not trade fractional shares
https://kernc.github.io/backtesting.py/#example


- Other backtesting framework: backtrader, zipline - both can do multi-asset trading
- Backtrader works with Pandas DataFrames, CSV, and real-time data feeds from Interactive Brokers, Oanda, and Visual Chart. 

Hypothesis
- Takes in a df from start to end, with all the ticker data (including those NA for sentiment)
- Enters trade at 549 (My information should backfill)
548	308.0	247.7006	247.7000	247.7000	247.7000	247.7000	1704291540000	8	2024-01-03 14:19:00	2024-01-03 14:19:00	0.156808
549	264.0	247.6105	247.6000	247.6000	247.6000	247.6000	1704291780000	9	2024-01-03 14:23:00	NaT	NaN
550	1157.0	247.5724	247.6000	247.5031	247.6001	247.5031	1704291840000	49	2024-01-03 14:24:00	NaT	NaN

In [65]:
# TODO: Shift the polarity down or the date up (the data with today's date is the data from ytd)
# TODO: How to add the number of positions to put? Or leverage?
# Assume your data includes 'Open', 'Close', and 'Sentiment' columns
# data = pd.read_csv('your_stock_data.csv', parse_dates=True, index_col='Date')

class SentimentStrategy(Strategy):
    """
    On today, it will use the information on the same row as today. After you gather
    all previous information, you'll need to put it to "today".
    Assumption: uses ascending data: goes from top to bottom row by row.
    """
    def init(self):
        super().init()
        # Initialize additional indicators here if needed
        self.long_pos_tsh = 0.4
        self.short_pos_tsh = -0.4
        self.trade_size = 100
        self.sl_pct = 0.005
        self.tp_pct = 0.005
        self.COL = 'cln_news_pol_sid_score'
    def next(self):
        
        print(f"-----{self.data['Datetime'][-1]}-----")
        # It doesn't display the first, only start from the next
        # print(self.data)
        curr_stm = self.data[self.COL][-1]
        # print(self.data['closest_date'][-1])

        cur_price = self.data['Close'][-1]
        # TODO: Position represents all trades

        # Strong signals
        if curr_stm > self.long_pos_tsh and self.position.is_short:
            self.position.close()
            self.buy(size=self.trade_size * 2, sl=(1 - self.sl_pct) * cur_price, tp=(1 + self.tp_pct) * cur_price)
            # self.buy(size)
            logging.warning(f"curr_stm > {self.long_pos_tsh} but position is short.")
        elif curr_stm < self.short_pos_tsh and self.position.is_long:
            self.position.close()
            self.sell(size=self.trade_size * 2, sl=(1 + self.sl_pct) * cur_price, tp=(1 - self.tp_pct) * cur_price)
            logging.warning(f"curr_stm < {self.short_pos_tsh} but position is long.")
        else:
            if curr_stm > 0.1:  # Threshold for going long
                self.buy(size=self.trade_size, sl=(1 - self.sl_pct) * cur_price, tp=(1 + self.tp_pct) * cur_price)
            elif curr_stm < -0.1:
                self.sell(size=self.trade_size, sl=(1 + self.sl_pct) * cur_price, tp=(1 - self.tp_pct) * cur_price)
            # for trade in self.trades:
            #     print(f'Trade entry price, time:{trade.entry_price}, {trade.entry_time}')
            #     print(f'Is long, PL: {trade.is_long}, {trade.pl}')

            

            # Checking trade size (adjust as needed)
            # trade_size = 1  # Example trade size, adjust based on your strategy
            stop_loss_pct = 0.1
            take_profit_pct = 0.2

            # Trading logic based on sentiment score
            # if curr_stm > 0.1:  # Threshold for going long
            #     self.buy(size=trade_size)
            #     # if not self.position or self.position.is_short:
            #     #     self.position.close()  # Close short position if any
            #     #     self.buy(size=trade_size)
            #     #     print(f"Buy on: {self.data.index[-1]}, Sentiment: {self.data['cln_news_pol_blob_score'][-1]}")
            # elif curr_stm < -0.1:  # Threshold for going short
            #     self.sell(size=trade_size)
            #     # if not self.position or self.position.is_long:
            #     #     self.position.close()  # Close long position if any
            #     #     self.sell(size=trade_size)
            #     #     # print(f"Sell on: {self.data.index[-1]}, Sentiment: {self.data.Sentiment[-1]}")

# Additional considerations
# 1. Slippage: Incorporate slippage into your trades, if your backtesting framework allows.
# 2. Execution Price: Decide whether to execute at the current day's close or the next day's open.
# 3. Risk Management: Implement risk management strategies like stop-loss orders.

# Running the backtest
bt = Backtest(data=merged2, 
              strategy=SentimentStrategy, 
              cash=10000, 
              margin=0.02,
              commission=.0,
              trade_on_close=False,
              hedging=True
              ) # TODO: Adjust commission
results = bt.run()
# TODO: 
# bt.optimize(maximise='SQN',
#             method='grid',
#             max_tries=None,
#             constraint=None,
#             return_heatmap=True,
#             return_optimization=False # For the method 'skopt
#             ) 
display(results)
bt.plot() # TODO: Can have filename, plot in html

# TODO: Change to use close instead of open, change html location

-----2024-01-02T09:39:00.000000000-----
-----2024-01-02T10:05:00.000000000-----
-----2024-01-02T10:08:00.000000000-----
-----2024-01-02T10:20:00.000000000-----
-----2024-01-02T10:21:00.000000000-----
-----2024-01-02T10:22:00.000000000-----
-----2024-01-02T10:23:00.000000000-----
-----2024-01-02T10:24:00.000000000-----
-----2024-01-02T10:25:00.000000000-----
-----2024-01-02T10:46:00.000000000-----
-----2024-01-02T11:01:00.000000000-----
-----2024-01-02T11:16:00.000000000-----
-----2024-01-02T11:29:00.000000000-----
-----2024-01-02T11:48:00.000000000-----
-----2024-01-02T12:02:00.000000000-----
-----2024-01-02T12:07:00.000000000-----
-----2024-01-02T12:09:00.000000000-----
-----2024-01-02T12:10:00.000000000-----
-----2024-01-02T12:11:00.000000000-----
-----2024-01-02T12:12:00.000000000-----
-----2024-01-02T12:13:00.000000000-----
-----2024-01-02T12:14:00.000000000-----
-----2024-01-02T12:32:00.000000000-----
-----2024-01-02T12:43:00.000000000-----
-----2024-01-02T12:51:00.000000000-----


/var/folders/nw/_dsfxl1x5wl1p_pdmrb1y1hh0000gn/T/ipykernel_72979/3949565118.py:78: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data=merged2,


Start                                     0.0
End                                    5208.0
Duration                               5208.0
Exposure Time [%]                   38.990209
Equity Final [$]                  11162.12265
Equity Peak [$]                   12396.91355
Return [%]                          11.621226
Buy & Hold Return [%]              -16.596235
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.202466
Avg. Drawdown [%]                    -1.42794
Max. Drawdown Duration                 1555.0
Avg. Drawdown Duration             178.708333
# Trades                                 64.0
Win Rate [%]                           46.875
Best Trade [%]                       6.827858
Worst Trade [%]                     -0.633326
Avg. Trade [%]                    

INFO:bokeh.io.state:Session output file 'SentimentStrategy.html' already exists, will be overwritten.
/opt/homebrew/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/homebrew/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p5525', ...)

Return in %
- Basic: -4.060443
- Only buy with strong signals: -3.495643
- SID: 11.621226

In [22]:
results.get('Return [%]')

-4.96074300000002